In [ ]:
!pip install mapply

# Import Library

In [1]:
# Import Library Pendukung untuk Proses Text Pre-processing
import re
import pandas as pd
import numpy as np
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from datetime import timedelta, datetime
import mapply #Library untuk mengaktifkan multi threading / multi core

In [2]:
mapply.init(
    n_workers=-1,
    chunk_size=100,
    max_chunks_per_worker=8,
    progressbar=False
)

In [3]:
# Import File Normalisasi
with open("./normalisasi.txt") as f:
    normalisasi = f.read().splitlines()
# normalisasi

In [4]:
# Import File Stopwords
with open("./stopword.txt") as f:
    stopwords = f.read().splitlines()
# stopwords

In [5]:
# Import Data Latih
df = pd.read_csv("datalatih.csv",sep=',')
df

,Timestamp,Email,Nama Mahasiswa,NIM,Semester,Golongan,Matakuliah,Minggu Ke,Tanggal Perkuliahan,Topik,Apa yang sudah anda Pelajari ?
0,5/25/2021 22:52:46,nevarida24@gmail.com,Nevarida sindyka diliana putri,E41180466,6,A,Organisasi dan Arsitektur Komputer,9,5/25/2021,Overview materi,Dalam perkuliahan hari ini pak angga memberika...
1,5/25/2021 19:18:31,shandy318@gmail.com,Shandy Maulana Yudantiar,E41180463,6,A,Organisasi dan Arsitektur Komputer,9,5/25/2021,Overview Materi,Dalam pertemuan ini mengingat kembali definisi...
2,5/25/2021 18:20:46,meilinniafa@gmail.com,Meilinnia Fortuna Astri,E41180229,6,A,Organisasi dan Arsitektur Komputer,9,5/25/2021,Overview Materi,Komputer berasal dari kata Computare yang arti...
3,5/25/2021 18:21:03,adynugrahaputra@gmail.com,Ady Nugraha Putra Ramadhan,E41180123,6,A,Organisasi dan Arsitektur Komputer,9,5/25/2021,Overview Materi,Pada Pertemuan Hari ini saya membuat suatu rin...
4,5/25/2021 14:37:22,refandika888@gmail.com,Refandra Bramandika Miftahul Akhyar,E41180449,6,A,Organisasi dan Arsitektur Komputer,9,5/25/2021,Overview,Saya mempelajari cara kerja perintah yang masu...
5,5/25/2021 20:01:52,dimasfajrul24@gmail.com,dimas Fajrul Falah,e41180458,6,A,Organisasi dan Arsitektur Komputer,9,5/25/2021,Review video Inside your computer - Bettina Bair,"dari video yang telah saya lihat dan amati, sa..."
6,5/25/2021 18:02:01,Ryanhartadi999@gmail.com,Ryan Hartadi,E41180111,6,A,Organisasi dan Arsitektur Komputer,9,5/25/2021,Overview Materi,Cara komputer bekerja ketika kita klik mouse m...
7,5/25/2021 20:48:38,dekos301199@gmail.com,Denny eko satrijo,E41180325,6,A,Organisasi dan Arsitektur Komputer,9,5/25/2021,Materi Overview,Organisasi personal personal komputer & Arsi...
8,5/25/2021 21:02:43,titikrosanti2000@gmail.com,Titik Rosanti,E41180124,6,A,Organisasi dan Arsitektur Komputer,9,5/25/2021,Apa itu organisasi komputer dan arsitektur,Organisasi komputer dan Arsitektur komputer me...
9,5/25/2021 21:17:07,Nafislestamanta@gmail.com,Nafis Hibatullah Lestamanta,E41180163,6,A,Organisasi dan Arsitektur Komputer,9,5/25/2021,Inside Your Computer,Organisasi komputer serta Arsitektur komputer ...


In [6]:
# Uji coba Normalisasi
term = ['organisasi', 'personal', 'personal', 'komputer', 'arsitektur', 'personal', 'komputer', 'yg', 'paham', 'ajar', 'ciri', 'personal', 'personal', 'komputer', 'simpul', 'video', 'gera', 'perintah', 'mouse', 'perintah', 'dasar', 'orang', 'user', 'buka', 'buah', 'aplikasi', 'perintah', 'mouse', 'masuk', 'perintah', 'dasar', 'mouse', 'tuju', 'cpu', 'cpu', 'perintah', 'proses', 'pergi', 'artimatika', 'karna', 'dasar', 'personal', 'personal', 'komputer', 'indera', 'yg', 'sanggup', 'proses', 'data', 'aritmatika', 'cpu', 'hasil', 'baca', 'perintah', 'masuk', 'dasar', 'mouse', 'cpu', 'hasil', 'yg', 'tampil', 'layar', 'monitor', 'personal', 'komputer', 'sinkron', 'pakai', 'perintah', 'yg', 'laku', 'si', 'aplikasi', 'buka']
temp = " ".join(term)
for w in normalisasi:
    try:
        w_ = w.split(',')
        if term.index(w_[0]):
            temp = [w_[1] if word == w_[0] else word for word in term]
    except:
        continue
# temp

# Buat Fungsi Preprocessing

In [7]:
# Tahapan Text Pre-processing
# Cleaning
def _cleaning(term):
    # Hapus pencatuman username    
    _term = re.sub(r"@[\w]*", "", term)
    # Hapus hyperlink  
    _term =  re.sub('((www\.[^\s]+)|(https?://[^\s]+))','URL', _term)
    # Hapus special character    
    _term = re.sub(r"[()\"#/@;:<>{}*`'+=~|.!?,]", "", _term)
    _term = re.sub('[^a-zA-Zа-яА-Я]+', ' ', _term)
    # Remove multiple space 
    _term = re.sub("\s\s+", " ", _term)
    # Remove emoticons
    _regrex_pattern = re.compile(pattern="["
                                    u"\U0001F600-\U0001F64F"  # emoticons
                                    u"\U0001F300-\U0001F5FF"  # symbols
                                    u"\U0001F680-\U0001F6FF"  # transport & map symbols
                                    u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                                    "]+", flags=re.UNICODE)
    return re.sub(_regrex_pattern, "", _term)
    
# Casefolding (lowercase)
def _casefolding(term):
    _term = term.lower()
    return _term

# Tokenizing (token)
def _tokenizing(term):
    if not isinstance(term, str): return
    return term.split()

# Normalisasi (Ubah kata alay atau typo)
def _normalisasi(term):
    temp = term
    for w in normalisasi:
        try:
            w_ = w.split(',')
            if term.index(w_[0]):
                temp = [w_[1] if word == w_[0] else word for word in term]
        except:
            continue
    return temp

# Stopword (hapus kata tidak penting (bag of words))
def _stopword(term):
    temp = [w for w in term if not w in stopwords]
    return temp

# Stemming (pengambilan kata dasar)
def _stemming(terms):
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    tweet = stemmer.stem(" ".join(terms))
    return tweet.split()

# Proses Text Pre-processing
def normalization(term): 
    cleaning = _cleaning(term)
    casefolding = _casefolding(cleaning)
    tokenizing = _tokenizing(casefolding)
    normalisasi = _normalisasi(tokenizing)
    stopword = _stopword(normalisasi)
    stemming = _stemming(stopword)
   
    return [cleaning, casefolding, tokenizing, normalisasi, stopword, stemming]

def callback(x):
    cleaning, casefolding, tokenizing, normalisasi, stopword, stemming = normalization(x)
    return pd.Series({'cleaning': cleaning, 'casefolding': casefolding, 'tokenizing': tokenizing,'normalisasi':normalisasi,'stopword': stopword,'stemming': stemming, 'cleaned': " ".join(stemming)})

# Import Data Training

In [8]:
df['Apa yang sudah anda Pelajari ?'].head()

0    Dalam perkuliahan hari ini pak angga memberika...
1    Dalam pertemuan ini mengingat kembali definisi...
2    Komputer berasal dari kata Computare yang arti...
3    Pada Pertemuan Hari ini saya membuat suatu rin...
4    Saya mempelajari cara kerja perintah yang masu...
Name: Apa yang sudah anda Pelajari ?, dtype: object

In [9]:
normalization = df['Apa yang sudah anda Pelajari ?'].mapply(callback)
df = pd.concat([df,normalization], axis=1, join='inner')

In [10]:
df['stemming']
df

,Timestamp,Email,Nama Mahasiswa,NIM,Semester,Golongan,Matakuliah,Minggu Ke,Tanggal Perkuliahan,Topik,Apa yang sudah anda Pelajari ?,cleaning,casefolding,tokenizing,normalisasi,stopword,stemming,cleaned
0,5/25/2021 22:52:46,nevarida24@gmail.com,Nevarida sindyka diliana putri,E41180466,6,A,Organisasi dan Arsitektur Komputer,9,5/25/2021,Overview materi,Dalam perkuliahan hari ini pak angga memberika...,Dalam perkuliahan hari ini pak angga memberika...,dalam perkuliahan hari ini pak angga memberika...,"[dalam, perkuliahan, hari, ini, pak, angga, me...","[dalam, perkuliahan, hari, ini, pak, angga, me...","[perkuliahan, angga, tugas, mengamati, merevie...","[kuliah, angga, tugas, amat, mereview, resume,...",kuliah angga tugas amat mereview resume vidio ...
1,5/25/2021 19:18:31,shandy318@gmail.com,Shandy Maulana Yudantiar,E41180463,6,A,Organisasi dan Arsitektur Komputer,9,5/25/2021,Overview Materi,Dalam pertemuan ini mengingat kembali definisi...,Dalam pertemuan ini mengingat kembali definisi...,dalam pertemuan ini mengingat kembali definisi...,"[dalam, pertemuan, ini, mengingat, kembali, de...","[dalam, pertemuan, ini, mengingat, kembali, de...","[pertemuan, definisi, infrastruktur, organisas...","[temu, definisi, infrastruktur, organisasi, ko...",temu definisi infrastruktur organisasi kompute...
2,5/25/2021 18:20:46,meilinniafa@gmail.com,Meilinnia Fortuna Astri,E41180229,6,A,Organisasi dan Arsitektur Komputer,9,5/25/2021,Overview Materi,Komputer berasal dari kata Computare yang arti...,Komputer berasal dari kata Computare yang arti...,komputer berasal dari kata computare yang arti...,"[komputer, berasal, dari, kata, computare, yan...","[komputer, berasal, dari, kata, computare, yan...","[komputer, berasal, computare, menghitung, bah...","[komputer, asal, computare, hitung, bahasa, ko...",komputer asal computare hitung bahasa komputer...
3,5/25/2021 18:21:03,adynugrahaputra@gmail.com,Ady Nugraha Putra Ramadhan,E41180123,6,A,Organisasi dan Arsitektur Komputer,9,5/25/2021,Overview Materi,Pada Pertemuan Hari ini saya membuat suatu rin...,Pada Pertemuan Hari ini saya membuat suatu rin...,pada pertemuan hari ini saya membuat suatu rin...,"[pada, pertemuan, hari, ini, saya, membuat, su...","[pada, pertemuan, hari, ini, saya, membuat, su...","[pertemuan, ringkasan, kesimpulan, definisi, k...","[temu, ringkas, simpul, definisi, komputer, ar...",temu ringkas simpul definisi komputer arsitekt...
4,5/25/2021 14:37:22,refandika888@gmail.com,Refandra Bramandika Miftahul Akhyar,E41180449,6,A,Organisasi dan Arsitektur Komputer,9,5/25/2021,Overview,Saya mempelajari cara kerja perintah yang masu...,Saya mempelajari cara kerja perintah yang masu...,saya mempelajari cara kerja perintah yang masu...,"[saya, mempelajari, cara, kerja, perintah, yan...","[saya, mempelajari, cara, kerja, perintah, yan...","[mempelajari, kerja, perintah, masuk, kedalam,...","[ajar, kerja, perintah, masuk, dalam, komputer...",ajar kerja perintah masuk dalam komputer prose...
5,5/25/2021 20:01:52,dimasfajrul24@gmail.com,dimas Fajrul Falah,e41180458,6,A,Organisasi dan Arsitektur Komputer,9,5/25/2021,Review video Inside your computer - Bettina Bair,"dari video yang telah saya lihat dan amati, sa...",dari video yang telah saya lihat dan amati say...,dari video yang telah saya lihat dan amati say...,"[dari, video, yang, telah, saya, lihat, dan, a...","[dari, video, yang, telah, saya, lihat, dan, a...","[video, lihat, amati, mempelajari, kerja, peri...","[video, lihat, amat, ajar, kerja, perintah, ma...",video lihat amat ajar kerja perintah masuk dal...
6,5/25/2021 18:02:01,Ryanhartadi999@gmail.com,Ryan Hartadi,E41180111,6,A,Organisasi dan Arsitektur Komputer,9,5/25/2021,Overview Materi,Cara komputer bekerja ketika kita klik mouse m...,Cara komputer bekerja ketika kita klik mouse m...,cara komputer bekerja ketika kita klik mouse m...,"[cara, komputer, bekerja, ketika, kita, klik, ...","[cara, komputer, bekerja, ketika, kita, klik, ...","[komputer, klik, mouse, mengirimkan, pesan, k

# Export Menjadi File CSV

In [11]:
# Export Hasil Text Pre-processing
filename = "Hasil_PreProcessing.csv"
df.to_csv(filename, index=False)